# Feature Construction

This script constructs the following features:
- Is First-time contributor field
- Second-order degree centrality
- Link intensity

These fields are added to the dataset, removing the fields they are composed of (in case they are aggregated features).
The final dataset is outputted to a `.csv` file.

## Dataset loading

In [27]:
import pandas as pd
import python_proj.utils.exp_utils as exp_utils

input_data_file_name = "non_ftc_data"

data_path_format = exp_utils.BASE_PATH + "/final_data/{data_file_name}.csv"
data_path = data_path_format.format(data_file_name=input_data_file_name)
print(f"{data_path=}")


df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)

print(f"{len(df)=}.")
print(f"{len(df.columns)=}\n")

df.describe()

data_path='/workspaces/msc_thesis/data//final_data/non_ftc_data.csv'
len(df)=1815972.
len(df.columns)=97



,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRIntegratorToSubmitter-In),FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRCommenterToSubmitter-In),FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRCommenterToCommenter-In),FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.IssueCommenterToCommenter-In),FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.IssueCommenterToSubmitter-In),...,SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter,SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter,EcosystemExperienceSubmitterIssueSubmissionCount,EcosystemExperienceSubmitterIssueCommentCount,DependencyEcosystemExperienceSubmitterIssueSubmissionCount,DependencyEcosystemExperienceSubmitterIssueCommentCount,NonDependencyEcosystemExperienceSubmitterIssueSubmissionCount,NonDependencyEcosystemExperienceSubmitterIssueCommentCount,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,...,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.474751e+08,5.198276e+06,3.807207e+03,3.190111e+04,4.947501e+00,4.264890e+01,1.382600e+01,1.539485e+01,1.021316e+01,1.022213e+01,...,4.025249e-01,7.930369e+00,2.615631e+00,1.785944e+01,1.743463e-01,8.645552e-01,2.274088e+00,1.570265e+01,1.854588e-01,1.355166e+00
std,1.004294e+08,8.100370e+06,8.071615e+03,1.399335e+05,7.283362e+01,2.501879e+03,3.539189e+02,4.735927e+02,3.782511e+02,2.043811e+02,...,3.021098e+00,7.431202e+01,7.985540e+00,6.700519e+01,1.580686e+00,1.186131e+01,7.064174e+00,6.131080e+01,2.446396e+00,1.667164e+01
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.807322e+07,4.631570e+05,1.000000e+02,5.290000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.371940e+08,1.620265e+06,5.480000e+02,9.799167e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.278026e+08,6.384100e+06,2.817000e+03,7.328350e+03,3.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,2.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
max,3.617756e+08,5.976123e+07,8.297600e+04,4.489490e+06,1.000000e+04,3.461280e+05,8.789200e+04,1.227920e+05,1.095040e+05,4.764000e+04,...,2.190000e+02,1.951100e+04,2.332000e+03,7.630000e+03,3.730000e+02,9.270000e+02,2.332000e+03,7.630000e+03,7.410000e+02,2.764000e+03


## Construct first-time contributor

This is done by merging the datasets generated by `sliding_window_2` and `sliding_window_3`.
The latter is cannot generate FTC status accurately because it's multithreaded.

In [28]:
def replace(
    df1: pd.DataFrame, df2: pd.DataFrame, match_column: str, overwritten_column: str
) -> pd.DataFrame:
    replace_data = df2[[match_column, overwritten_column]]

    old_counts = df1[overwritten_column].value_counts()

    df1 = df1.merge(
        replace_data, how="left", on=match_column, suffixes=("_incorrect", "")
    )
    old_key = f"{overwritten_column}_incorrect"
    df1 = df1.drop(old_key, axis=1)

    new_counts = df1[overwritten_column].value_counts()

    diff = new_counts - old_counts
    print(f"Impact overwriting '{overwritten_column}':\n{diff}")

    return df1


other_data_name = "ftc_data"
other_data_path = f"{exp_utils.BASE_PATH}/final_data/{other_data_name}.csv"
print(f"{other_data_path=}\n")

other_df: pd.DataFrame = pd.read_csv(other_data_path)

first_time_contributor_key = "SubmitterIsFirstTimeContributor"
df = replace(df, other_df, "ID", first_time_contributor_key)

del other_df

other_data_path='/workspaces/msc_thesis/data//final_data/ftc_data.csv'

Impact overwriting 'SubmitterIsFirstTimeContributor':
SubmitterIsFirstTimeContributor
False    215739
True    -215739
Name: count, dtype: int64


## Construct second-order degree centrality

Calculates 2nd-order degree centrality by aggregating the 2nd-order in-/out-degree values weighing them according to their prevalence, using normalized reciprocal of the edge count; i.e., such that more common edges are weighted less.

_Note, the fields are named 1st-order degree centrality. This is a mistake, but I don't want to rewrite all the data retrieval code._

In [29]:
import regex as re

fo_in_exp = r".*FirstOrderDegree.*In\).*"
so_degree_in_features = list(
    [field for field in df.columns if re.match(fo_in_exp, field)]
)

fo_out_exp = r".*FirstOrderDegree.*Out\).*"
so_degree_out_features = list(
    [field for field in df.columns if re.match(fo_out_exp, field)]
)

print(f"{len(so_degree_in_features)=}, {len(so_degree_out_features)=}.")

len(so_degree_in_features)=25, len(so_degree_out_features)=25.


In [30]:
import json

# Calculates the edge weights.

# NOTE: There's nothing automated about these numbers;
# I copy-pasted them from the terminal after running ``sliding_window_2.py`` on the dataset.
edge_counts = {
    "PRIntegratorToSubmitter": 1043559,
    "PRCommenterToSubmitter": 2808889,
    "PRCommenterToCommenter": 24595282,
    "IssueCommenterToCommenter": 73500466,
    "IssueCommenterToSubmitter": 5806894,
}
total_edges = sum(edge_counts.values())

# ahp_weights = {
#     "PRIntegratorToSubmitter": 0.09,
#     "PRCommenterToSubmitter": 0.16,
#     "PRCommenterToCommenter": 0.30,
#     "IssueCommenterToCommenter": 0.16,
#     "IssueCommenterToSubmitter": 0.30
# }


def calculate_weight(value):
    """Calculates normalized complement of the weight."""
    percentage_complement = 1 - (value / (total_edges))
    normalized_perc_compl = percentage_complement / (len(edge_counts) - 1)
    return normalized_perc_compl


edge_weights = {
    key: calculate_weight(value)  # * ahp_weights[key]
    for key, value in edge_counts.items()
}
total_weight = sum(edge_weights.values())
edge_weights = {key: value / total_weight for key, value in edge_weights.items()}

print("Edge weights:\n", json.dumps(edge_weights, indent=4))

Edge weights:
 {
    "PRIntegratorToSubmitter": 0.24757886379195637,
    "PRCommenterToSubmitter": 0.24348316399717174,
    "PRCommenterToCommenter": 0.19293707610471117,
    "IssueCommenterToCommenter": 0.07947333160781547,
    "IssueCommenterToSubmitter": 0.23652756449834528
}


In [31]:
from functools import partial
import itertools


# Feature construction
base_field_name = (
    "FirstOrderDegreeCentralityV2({connecting_edge}.{experience_edge}-{direction})"
)


def calculate_weighted_so_centrality(series: pd.Series, direction: str):
    """Aggregates the 2nd-order degrees into one centrality measure."""

    centrality = 0.0
    partial_field_name = partial(base_field_name.format, direction=direction)

    # Adds centrality contribution per pair of edges.
    for connecting_edge, experience_edge in itertools.product(
        edge_weights.keys(), edge_weights.keys()
    ):
        connection_weight = edge_weights[connecting_edge]
        experience_weight = edge_weights[experience_edge]

        current_field = partial_field_name(
            connecting_edge=connecting_edge, experience_edge=experience_edge
        )
        experience = series[current_field]

        centrality += connection_weight * experience_weight * experience

    return centrality


weighted_in_fod_key = "WeightedFirstOrderInDegreeCentrality"
df[weighted_in_fod_key] = df[so_degree_in_features].apply(
    partial(calculate_weighted_so_centrality, direction="In"), axis=1
)

weighted_out_fod_key = "WeightedFirstOrderOutDegreeCentrality"
df[weighted_out_fod_key] = df[so_degree_out_features].apply(
    partial(calculate_weighted_so_centrality, direction="Out"), axis=1
)

centr_fields = [weighted_in_fod_key, weighted_out_fod_key]
df[centr_fields].describe()

,WeightedFirstOrderInDegreeCentrality,WeightedFirstOrderOutDegreeCentrality
count,1.815972e+06,1.815972e+06
mean,5.062849e+01,5.159898e+01
std,1.718804e+03,2.461762e+03
min,0.000000e+00,0.000000e+00
25%,6.129529e-02,0.000000e+00
50%,1.398842e+00,7.980457e-01
75%,6.308026e+00,4.222920e+00
max,1.035309e+06,1.488839e+06


In [32]:
features_to_drop = [*so_degree_in_features, *so_degree_out_features]
print(f"{len(features_to_drop)=}")
print(features_to_drop)
df = df.drop(features_to_drop, axis=1)

len(features_to_drop)=50
['FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRIntegratorToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRCommenterToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.PRCommenterToCommenter-In)', 'FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.IssueCommenterToCommenter-In)', 'FirstOrderDegreeCentralityV2(PRIntegratorToSubmitter.IssueCommenterToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToSubmitter.PRIntegratorToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToSubmitter.PRCommenterToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToSubmitter.PRCommenterToCommenter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToSubmitter.IssueCommenterToCommenter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToSubmitter.IssueCommenterToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToCommenter.PRIntegratorToSubmitter-In)', 'FirstOrderDegreeCentralityV2(PRCommenterToCommenter

## Construct Link intensity

Calculates link intensity by aggregating the individual fields, weighting them using the normalized reciprocal of the edge count (the same weights as used for 2nd-order degree centrality).

In [33]:
sub_to_int_fields = [
    "SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator",
    "SharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator",
    "SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter",
    "SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter",
    "SharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator",
]

int_to_sub_fields = [
    "SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter",
    "SharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter",
    "SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter",
    "SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter",
    "SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter",
]

In [34]:
def calculate_intensity(
    series: pd.Series, source_fields: list[str], source_weights: list[str]
) -> int:
    """Calculates weighted sum of link intensity fields."""
    return sum(
        [series[field] * weight for field, weight in zip(source_fields, source_weights)]
    )


its_key = "IntegratorToSubmitterLinkIntensity"
df[its_key] = df.apply(
    partial(
        calculate_intensity,
        source_fields=int_to_sub_fields,
        source_weights=edge_weights.values(),
    ),
    axis=1,
)

tsi_key = "SubmitterToIntegratorLinkIntensity"
df[tsi_key] = df.apply(
    partial(
        calculate_intensity,
        source_fields=sub_to_int_fields,
        source_weights=edge_weights.values(),
    ),
    axis=1,
)

df[[its_key, tsi_key]].describe()

,IntegratorToSubmitterLinkIntensity,SubmitterToIntegratorLinkIntensity
count,1.815972e+06,1.815972e+06
mean,3.331470e+00,4.316357e+00
std,2.971514e+01,3.035006e+01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,7.947333e-02,6.582846e-01
max,4.436984e+03,4.238017e+03


In [35]:
features_to_drop = [*sub_to_int_fields, *int_to_sub_fields]
print(f"{len(features_to_drop)=}")
print(features_to_drop)
df = df.drop(features_to_drop, axis=1)

len(features_to_drop)=10
['SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator', 'SharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator', 'SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter', 'SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter', 'SharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator', 'SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter', 'SharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter', 'SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter', 'SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter', 'SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter']


## Save dataset to file

In [36]:
output_data_file_name = "dataset_with_constructed"
output_path = data_path_format.format(data_file_name=output_data_file_name)
print(f"Outputting to '{output_path}'")

df.to_csv(output_path, index=False)

df.describe()

Outputting to '/workspaces/msc_thesis/data//final_data/dataset_with_constructed.csv'


,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,ControlIntraProjectPullRequestExperienceOfIntegrator,IntraProjectSubmitterPullRequestSubmissionCount,IntraProjectSubmitterPullRequestSuccessRate,IntraProjectSubmitterPullRequestCommentCount,EcosystemExperienceSubmitterPullRequestSuccessRate,...,DependencyEcosystemExperienceSubmitterIssueSubmissionCount,DependencyEcosystemExperienceSubmitterIssueCommentCount,NonDependencyEcosystemExperienceSubmitterIssueSubmissionCount,NonDependencyEcosystemExperienceSubmitterIssueCommentCount,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount,WeightedFirstOrderInDegreeCentrality,WeightedFirstOrderOutDegreeCentrality,IntegratorToSubmitterLinkIntensity,SubmitterToIntegratorLinkIntensity
count,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,...,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06,1.815972e+06
mean,1.474751e+08,5.198276e+06,3.807207e+03,3.190111e+04,4.947501e+00,1.932087e+02,1.679350e+01,5.352468e-01,2.170048e+01,3.702776e-01,...,1.743463e-01,8.645552e-01,2.274088e+00,1.570265e+01,1.854588e-01,1.355166e+00,5.062849e+01,5.159898e+01,3.331470e+00,4.316357e+00
std,1.004294e+08,8.100370e+06,8.071615e+03,1.399335e+05,7.283362e+01,4.827151e+02,5.548605e+01,4.623260e-01,6.563799e+01,4.472133e-01,...,1.580686e+00,1.186131e+01,7.064174e+00,6.131080e+01,2.446396e+00,1.667164e+01,1.718804e+03,2.461762e+03,2.971514e+01,3.035006e+01
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.807322e+07,4.631570e+05,1.000000e+02,5.290000e+01,1.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.129529e-02,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.371940e+08,1.620265e+06,5.480000e+02,9.799167e+02,1.000000e+00,3.000000e+01,2.000000e+00,8.000000e-01,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.398842e+00,7.980457e-01,0.000000e+00,0.000000e+00
75%,2.278026e+08,6.384100e+06,2.817000e+03,7.328350e+03,3.000000e+00,1.410000e+02,1.700000e+01,1.000000e+00,1.500000e+01,9.186047e-01,...,0.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,6.308026e+00,4.222920e+00,7.947333e-02,6.582846e-01
max,3.617756e+08,5.976123e+07,8.297600e+04,4.489490e+06,1.000000e+04,4.281000e+03,1.703000e+03,1.000000e+00,2.038000e+03,1.000000e+00,...,3.730000e+02,9.270000e+02,2.332000e+03,7.630000e+03,7.410000e+02,2.764000e+03,1.035309e+06,1.488839e+06,4.436984e+03,4.238017e+03


In [38]:
import datetime

print(datetime.datetime.now())

2024-01-29 13:46:02.478242
